In [32]:
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from skmob.utils import utils
from math import pi, asin, sin, cos, sqrt

pd.set_option('display.max_columns', 500)



%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

In [33]:
def distance_earth_km(src, dest):

    lat1, lat2 = src['lat']*pi/180, dest['lat']*pi/180
    lon1, lon2 = src['lon']*pi/180, dest['lon']*pi/180
    dlat, dlon = lat1-lat2, lon1-lon2

    ds = 2 * asin(sqrt(sin(dlat/2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2.0) ** 2))
    return 6371.01 * ds

In [34]:
def compute_distance_matrix(tess):

    lats_lngs = tess.geometry.apply(utils.get_geom_centroid, args=[True]).values
    
    distance_matrix = np.zeros((len(tess),len(tess)))

    for i in range(0,len(tess)):
        for j in range(0,len(tess)):
            if i != j:
                d = distance_earth_km({'lat':lats_lngs[i][0],'lon':lats_lngs[i][1]},
                                         {'lat':lats_lngs[j][0],'lon':lats_lngs[j][1]})
                distance_matrix[i,j] = d
                
    return distance_matrix

In [3]:
shape_file_land = gpd.read_file("Boundaries - Community Areas (current).geojson")
shape_file_land_filtered = shape_file_land[shape_file_land['community'].
                                           isin(['LOOP', 'NEAR SOUTH SIDE', 'NEAR NORTH SIDE', 'NEAR WEST SIDE', 'LOWER WEST SIDE', 'WEST TOWN'])]
shape_file_land_filtered

,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,geometry
24,WEST TOWN,0,127562904.597,0,24,24,0,0,55203.7186956,"MULTIPOLYGON (((-87.65686 41.91078, -87.65685 ..."
28,NEAR WEST SIDE,0,158492466.554,0,28,28,0,0,53003.5929406,"MULTIPOLYGON (((-87.63759 41.88623, -87.63765 ..."
32,LOWER WEST SIDE,0,81550723.6377,0,31,31,0,0,43229.3727036,"MULTIPOLYGON (((-87.63516 41.85772, -87.63532 ..."
33,NEAR SOUTH SIDE,0,49769639.4541,0,33,33,0,0,45053.1651038,"MULTIPOLYGON (((-87.60914 41.84469, -87.60938 ..."
36,NEAR NORTH SIDE,0,76675895.9728,0,8,8,0,0,57293.1649516,"MULTIPOLYGON (((-87.62446 41.91157, -87.62459 ..."
37,LOOP,0,46335565.4586,0,32,32,0,0,39428.6729962,"MULTIPOLYGON (((-87.60949 41.88940, -87.60949 ..."


In [4]:
meters = 1405
tesselletion = tilers.tiler.get("squared", meters=meters, base_shape=shape_file_land_filtered)
plot_gdf(tesselletion, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 9) 

In [6]:
tesselletion;

In [7]:
len(tesselletion)

64

In [37]:
dist = compute_distance_matrix(tesselletion)
dist

array([[0.        , 1.04457947, 2.08900526, ..., 8.92751626, 8.42290739,
        8.35724999],
       [1.04457947, 0.        , 1.04442579, ..., 9.34506394, 8.6144561 ,
        8.42166846],
       [2.08900526, 1.04442579, 0.        , ..., 9.85599213, 8.9248904 ,
        8.61318888],
       ...,
       [8.92751626, 9.34506394, 9.85599213, ..., 0.        , 2.08992727,
        3.13466041],
       [8.42290739, 8.6144561 , 8.9248904 , ..., 2.08992727, 0.        ,
        1.04473314],
       [8.35724999, 8.42166846, 8.61318888, ..., 3.13466041, 1.04473314,
        0.        ]])

In [38]:
np.save("dist_mat.npy", dist)